# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [13]:
import warnings
warnings.filterwarnings('ignore')
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import warnings 

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [14]:
datafile =pd.read_csv ("../output_data/mycitydata.csv")

datafile.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,1526038,Atbasar,100,KZ,1603573864,97,51.80,68.33,32.49,23.73
1,5122534,Jamestown,90,US,1603573087,75,42.10,-79.24,44.01,8.05
2,2152659,Port Macquarie,50,AU,1603573864,99,-31.43,152.92,66.99,1.99
3,5746545,Portland,75,US,1603573076,57,45.52,-122.68,51.01,18.34
4,4031574,Provideniya,76,RU,1603573865,89,64.38,-173.30,33.82,15.21


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [15]:
#Configure Heatmap
gmaps.configure(api_key=g_key)

# Store 'Lat' and 'Lng' into  locations 
locations = datafile[["Lat", "Lng"]].astype(float)
humidity = datafile["Humidity"].astype(float)

fig = gmaps.figure()
# create map
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 2)
# add layer
fig.add_layer(heat_layer)
# show map
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [16]:
new_df = pd.DataFrame(datafile, columns = ['City', 'Max Temp','Cloudiness','Humidity','Lat','Lng','Country'])
new_df.head()

,City,Max Temp,Cloudiness,Humidity,Lat,Lng,Country
0,Atbasar,32.49,100,97,51.80,68.33,KZ
1,Jamestown,44.01,90,75,42.10,-79.24,US
2,Port Macquarie,66.99,50,99,-31.43,152.92,AU
3,Portland,51.01,75,57,45.52,-122.68,US
4,Provideniya,33.82,76,89,64.38,-173.30,RU


In [17]:
# City that has the weather of my choice
hotel_df = new_df.loc[(new_df['Max Temp']>56) & (new_df['Max Temp']<80) & (new_df['Cloudiness']>0) 
                          & (new_df['Cloudiness']<40) & (new_df['Humidity']<30)]

hotel_df

,City,Max Temp,Cloudiness,Humidity,Lat,Lng,Country
190,Montrose,70.00,1,19,38.42,-108.27,US
203,Saint George,79.00,1,28,37.10,-113.58,US
353,Pueblo,78.01,1,27,38.25,-104.61,US
534,Nālūt,71.55,17,26,30.33,10.85,LY


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [18]:
# Hotel variable 
hotel_df["Hotel Name"] = ""
hotel_df

,City,Max Temp,Cloudiness,Humidity,Lat,Lng,Country,Hotel Name
190,Montrose,70.00,1,19,38.42,-108.27,US,
203,Saint George,79.00,1,28,37.10,-113.58,US,
353,Pueblo,78.01,1,27,38.25,-104.61,US,
534,Nālūt,71.55,17,26,30.33,10.85,LY,


In [19]:
for index, row in hotel_df.iterrows():
    lat =  row["Lat"]
    lng =  row["Lng"]
    
    #check to see if coor is pulled
    #print((f"{lat},{lng}"))
    
    # geocoordinates
    target_coordinates = (f"{lat},{lng}")
    #target_search = "Chinese"
    target_radius = 5000
    target_type = "lodging"

    # set up a parameters dictionary
    params = {
            "location":target_coordinates,
            "radius": target_radius,
            "type": target_type,
            "key": g_key
    }
    
    
    #print(params)
    
    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # run a request using my params dictionary
    hotel_resp = requests.get(base_url, params=params).json()

    # print pretty data
    #print(json.dumps(hotel_resp, indent=4, sort_keys=True))
                           
    try: 
  
       hotel_df.loc[index,'Hotel Name'] =hotel_resp["results"][0]["name"]
    except: 
       print("Hotel Not Found")
print("------End Run Code---------")


Hotel Not Found
Hotel Not Found
------End Run Code---------


In [24]:
# Clean dataset 
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
hotel_df.dropna(subset=['Hotel Name'], inplace=True)
hotel_df

,City,Max Temp,Cloudiness,Humidity,Lat,Lng,Country,Hotel Name
203,Saint George,79.00,1,28,37.10,-113.58,US,Best Western Plus Abbey Inn
353,Pueblo,78.01,1,27,38.25,-104.61,US,Hampton Inn & Suites Pueblo-Southgate


In [25]:
#Configure My Weather - Hotel Heatmap
# Create Hotel symbol layer
gmaps.configure(api_key=g_key)



# Store 'Lat' and 'Lng' into  locations 
locations = hotel_df[["Lat", "Lng"]].astype(float)
humidity = hotel_df["Humidity"].astype(float)


figMap = gmaps.figure()
# create map
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 2)


# add layer
figMap.add_layer(heat_layer)

# show map
figMap

Figure(layout=FigureLayout(height='420px'))

In [26]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]


In [28]:
# Add marker layer ontop of heat map
figMap = gmaps.figure(map_type='SATELLITE')
hotelmarkers = gmaps.marker_layer(locations,info_box_content=hotel_info)

figMap.add_layer(hotelmarkers)

figMap



Figure(layout=FigureLayout(height='420px'))

In [ ]:
results_path = 'MapHotel.png'
#print(results_path)
plt.savefig("../Images/results_path")